# Solução dos exercícios da Lista 2 de Otimização Linear

Professora Asla Medeiros e Sá

Tainá Isabela Queiroz Drumond - IMPA Tech

Nesse notebook, resolvo os exercícios propostos para a Lista 2 da disciplina de Otimização Linear do IMPA Tech.

In [190]:
import pulp as pl
import numpy as np
import pandas as pd
import altair as alt

## Exercício 1

Nesse exercício, buscamos resolver um problema de regressão linear utilizando programação linear.

### Item (a)

#### Enunciado:

Formule programas lineares (PLs) cujas soluções ótimas minimizem 

1. A soma dos resíduos absolutos da reta aproximada para os dados
$$\min_{a} \sum_{i=1}^n \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$$
2. O resíduo absoluto máximo
$$\min_{a} \max_{i} \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$$

Depois, generalize o modelo para permitir o ajuste para polinômios gerais
$$y = a_k x^k + a_{k-1} x^{k-1} + ... + a_1 x + a_0$$

#### Solução:

Nesse problema, nossas variáveis de decisão são os coeficientes $a_i$.

##### 1 - Soma dos resíduos absolutos

Precisamos linearizar os termos do somatório, já que o módulo não é uma operação linear. Para isso, vamos criar novas variáveis $z_i$ e minimizar o seguinte somatório:

$$\min_{a} \sum_{i=1}^n z_i$$

Onde

$z_i = \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$

Agora, é necessário linearizar as restrições sobre os $z_i$.

Perceba que, se $c=\bigl|\, a-b \,\bigr|$, então $c \geq a-b$ e $c \geq b-a$. Logo, podemos estabelecer as seguintes restrições:

$z_i \geq y_i - (a_1 x_i + a_0)$

$z_i \geq (a_1 x_i + a_0) - y_i$

Como o nosso programa minimiza o somatório dos $z_i$, a igualdade será estabelecida para alguma das duas restrições acima, chegando a $z_i = \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$.

##### 2 - Resíduo absoluto máximo

Para linearizar o problema, minimizaremos a variável $z$ onde
$$z = \max_{i} \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$$

Com a restrição de 

$z \geq y_i - (a_1 x_i + a_0)$

$z \geq (a_1 x_i + a_0) - y_i$

para todo $i = 1, ..., n$.

Assim, z será limitado inferiormente por todos os resíduos absolutos e, em particular, pelo maior deles. Como o problema minimiza z, chegaremos em $z = \min_{a} \max_{i} \bigl|\, y_i - (a_1 x_i + a_0) \,\bigr|$.

##### 3 - Generalizando para polinômios gerais

Tanto no caso da soma dos resíduos absolutos quanto no caso do resíduo absoluto máximo, só precisamos de mais variáveis decisão para os coeficientes adicionais e 

#### Modelagem computacional

In [191]:
def linear_regression_absolute_residual_sum(x, y):
    # Criação do problema
    lp = pl.LpProblem("Linear_Regression", pl.LpMinimize)

    n = len(x)
    
    # Variáveis de decisão (coeficientes da aproximação linear)
    a0 = pl.LpVariable("a0")
    a1 = pl.LpVariable("a1")
    z = pl.LpVariable.dicts("z", range(n))

    # Função objetivo
    lp += pl.lpSum(z), "Absolute_Residual_Sum"

    # Restrições
    for i in range(n):
        lp += z[i] - (x[i] * a1) - a0 + y[i] >= 0
        lp += z[i] + (x[i] * a1) + a0 - y[i] >= 0

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    return lp.variables()[0].varValue, lp.variables()[1].varValue

### Item (b)

Vamos gerar alguns dados em R² para testar o ajuste:

In [192]:
ex1b_n = 100
 
# Sorteio de x ~ U(0, 10)
ex1b_x = np.random.uniform(0, 10, ex1b_n)

# Erro aleatório ~ N(0, 2)
ex1b_erro = np.random.normal(0, 2, ex1b_n)

# Geração do y com ruído
ex1b_y = 5 * ex1b_x + 2 + ex1b_erro

# Dataframe dos dados gerados
ex1b_df = pd.DataFrame({"x": ex1b_x, "y": ex1b_y})

# Gráfico dos pontos gerados
ex1b_pontos = alt.Chart(ex1b_df).mark_point().encode(
    x="x",
    y="y"
).properties(
    title="Amostra de pontos de uma reta"
)

ex1b_pontos

alt.Chart(...)

Agora, realizamos o ajuste com a modelagem feita no item anterior:

In [193]:
ex1b_a0, ex1b_a1 = linear_regression_absolute_residual_sum(ex1b_x, ex1b_y)

# Geração de pontos no polinômio ajustado
ex1b_x_reta = np.linspace(0, 10, 100) 
ex1b_y_reta = ex1b_a1 * ex1b_x_reta + ex1b_a0

# Gráfico da reta ajustada
ex1b_df_reta = pd.DataFrame({"x": ex1b_x_reta, "y": ex1b_y_reta})

ex1b_reta = alt.Chart(ex1b_df_reta).mark_line(color="red").encode(
    x="x",
    y="y"
).properties(
    title="Ajuste de pontos em uma reta"
)

ex1b_reta + ex1b_pontos

Status: Optimal


alt.LayerChart(...)

### Item (c)

Adaptaremos o problema para a regressão a um polinômio não linear. Para isso, basta alterar as restrições para a variável $z$ para acomodar um polinômio de grau superior. Aqui, modelei um ajuste para um polinômio de grau 3. Nesse cenário, as restrições são:

$z \geq y_i - (a_3 x_i^3 + a_2 x_i^2 + a_1 x_i + a_0)$

$z \geq (a_3 x_i^3 + a_2 x_i^2 + a_1 x_i + a_0) - y_i$

In [194]:
def poly_approximation(x, y):
    # Criação do problema
    lp = pl.LpProblem("Polynomial_Approximation", pl.LpMinimize)

    n = len(x)
    
    # Variáveis de decisão (coeficientes da aproximação)
    a0 = pl.LpVariable("a0")
    a1 = pl.LpVariable("a1")
    a2 = pl.LpVariable("a2")
    a3 = pl.LpVariable("a3")
    z = pl.LpVariable.dicts("z", range(n))

    # Função objetivo
    lp += pl.lpSum(z), "Absolute_Residual_Sum"

    # Restrições
    for i in range(n):
        lp += z[i] - ((a3 * x[i]**3) + (a2 * x[i]**2) + (a1 * x[i]) + a0) + y[i] >= 0
        lp += z[i] + ((a3 * x[i]**3) + (a2 * x[i]**2) + (a1 * x[i]) + a0) - y[i] >= 0

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    return lp.variables()[0].varValue, lp.variables()[1].varValue, lp.variables()[2].varValue, lp.variables()[3].varValue

Gerando dados para testar a aproximação:

In [195]:
ex1c_n = 100
 
# Sorteio de x ~ U(0, 10)
ex1c_x = np.random.uniform(0, 10, ex1c_n)

# Erro aleatório ~ N(0, 2)
ex1c_erro = np.random.normal(0, 5, ex1c_n)

# Geração do y com ruído
ex1c_y = 0.6 * ex1c_x**3 + ex1c_x**2 - 9 * ex1c_x + 2 + ex1c_erro

# Dataframe dos dados gerados
ex1c_df = pd.DataFrame({"x": ex1c_x, "y": ex1c_y})

# Gráfico dos pontos gerados
ex1c_pontos = alt.Chart(ex1c_df).mark_point().encode(
    x="x",
    y="y"
).properties(
    title="Amostra de pontos em um polinômio"
)

ex1c_pontos

alt.Chart(...)

Calculando a regressão:

In [196]:
ex1c_a0, ex1c_a1, ex1c_a2, ex1c_a3 = poly_approximation(ex1c_x, ex1c_y)
print(ex1c_a0, ex1c_a1, ex1c_a2, ex1c_a3)

# Geraçaõ de pontos no polinômio ajustado
ex1c_x_reta = np.linspace(0, 10, 100) 
ex1c_y_reta = ex1c_a3 * ex1c_x_reta**3 + ex1c_a2 * ex1c_x_reta**2 + ex1c_a1 * ex1c_x_reta + ex1c_a0

# Gráfico do polinômio ajustado
ex1c_df_reta = pd.DataFrame({"x": ex1c_x_reta, "y": ex1c_y_reta})

ex1c_reta = alt.Chart(ex1c_df_reta).mark_line(color="red").encode(
    x="x",
    y="y"
).properties(
    title="Ajuste de pontos em um polinômio"
)

ex1c_reta + ex1c_pontos

Status: Optimal
2.7522375 -9.7028032 1.1732219 0.59003427


alt.LayerChart(...)

### Item (d)

Neste item vamos aumentar a dimensão do problema, isto é, considerar um número maior de variáveis.

Para isso, precisamos aumentar o número de variáveis de decisão para os coeficientes de acordo com o número de preditores, e ajustar as restrições para $z$.

Se temos $d$ preditores, precisamos de $d+1$ coeficientes $a_0, a_1, ..., a_d$.

Então, o código fica da seguinte forma:

In [197]:
def linear_regression_dim(X, y):
    n, d = X.shape
    
    # Criação do problema
    lp = pl.LpProblem("Linear_Regression", pl.LpMinimize)
    
    # Variáveis de decisão
    a0 = pl.LpVariable("a0")  # intercepto
    a = pl.LpVariable.dicts("a", range(d))  # coeficientes
    z = pl.LpVariable.dicts("z", range(n), lowBound=0)

    # Função objetivo
    lp += pl.lpSum(z[i] for i in range(n)), "Absolute_Residual_Sum"

    # Restrições
    for i in range(n):
        func = a0 + pl.lpSum(a[j] * X[i, j] for j in range(d))
        lp += z[i] - (y[i] - func) >= 0
        lp += z[i] + (y[i] - func) >= 0

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    a0_val = a0.varValue
    a_vals = np.array([a[j].varValue for j in range(d)])
    print("Status:", pl.LpStatus[lp.status])
    return a0_val, a_vals


Aplicação em pontos gerados:

In [198]:
ex1d_n = 100

# X com três preditores
ex1d_X = np.random.uniform(0, 10, (ex1d_n, 3))

# Erro aleatório ~ N(0, 2)
ex1d_erro = np.random.normal(0, 2, ex1d_n)

# Geração do y com coeficientes reais: y = 4*X1 - 2*X2 + 3*X3 + 5 + erro
ex1d_y = 4*ex1d_X[:,0] - 2*ex1d_X[:,1] + 3*ex1d_X[:,2] + 5 + ex1d_erro

# Ajuste
ex1d_a0, ex1d_a = linear_regression_dim(ex1d_X, ex1d_y)

print("Intercepto:", ex1d_a0)
print("Coeficientes:", ex1d_a)

Status: Optimal
Intercepto: 4.618725
Coeficientes: [ 4.0260246 -1.9701715  3.0125403]


## Exercício 2

Applied Mathematical Programming | Capítulo 9 - Exercício 5

### Item (a)

#### Enunciado

Reformule o problema a seguir como uma problema linear inteiro equivalente:

Maximizar:
$z = x_1 + 2x_2$

Sujeito a:
- $x_1 + x_2 \le 8$
- $-x_1 + x_2 \le 2$
- $x_1 - x_2 \le 4$
- $x_2 \ge 0$ e inteiro

$x_1 \in \{0, 1, 4, 6\}$

#### Solução

Criaremos variáveis de decisão binárias $y_0, y_1, y_4$ e $y_6$ para modelar o conjunto de valores possíveis para $x_1$. Para isso, temos as seguintes restrições:
- $y_0 + y_1 + y_4 + y_6 = 1$
- $x_1 = 0y_0 + 1y_1 + 4y_4 + 6y_6 \Rightarrow y_1 + 4y_4 + 6y_6 - x_1 = 0$ 


In [199]:
# Criação do problema
ex2a_lp = pl.LpProblem("Integer_Problem", pl.LpMaximize)

# Variáveis de decisão
ex2a_x1 = pl.LpVariable("x1")
ex2a_x2 = pl.LpVariable("x2", lowBound=0, cat="Integer")
ex2a_y0 = pl.LpVariable("y0", cat="Binary")
ex2a_y1 = pl.LpVariable("y1", cat="Binary")
ex2a_y4 = pl.LpVariable("y4", cat="Binary")
ex2a_y6 = pl.LpVariable("y6", cat="Binary")

# Função objetivo
ex2a_lp += ex2a_x1 + 2*ex2a_x2

# Restrições
ex2a_lp += ex2a_y0 + ex2a_y1 + ex2a_y4 + ex2a_y6 == 1
ex2a_lp += ex2a_y1 + 4*ex2a_y4 + 6*ex2a_y6 - ex2a_x1 == 0
ex2a_lp += ex2a_x1 + ex2a_x2 <= 8
ex2a_lp += -ex2a_x1 + ex2a_x2 <= 2
ex2a_lp += ex2a_x1 - ex2a_x2 <= 4

# Resolver
ex2a_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex2a_lp.status])
for var in ex2a_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex2a_lp.objective))

Status: Optimal
x1 = 4.0
x2 = 4.0
y0 = 0.0
y1 = 0.0
y4 = 1.0
y6 = 0.0
z = 12.0


### Item (b)

#### Enunciado

Como resolver o item (a) se a função objetivo for alterada para:

Maximizar: z = $x_1^2 + 2x_2$

#### Solução

Podemos criar uma outra variável para guardar o valor de $x_1^2$ da seguinte forma:

$w = 0y_0 + 1y_1 + 16y_4 + 36y_6$

Assim, alteramos nossa função objetivo para:

Maximizar: z = $w + 2x_2$

In [200]:
# Criação do problema
ex2b_lp = pl.LpProblem("Integer_Problem", pl.LpMaximize)

# Variáveis de decisão
ex2b_x1 = pl.LpVariable("x1")
ex2b_x2 = pl.LpVariable("x2", lowBound=0, cat="Integer")
ex2b_y0 = pl.LpVariable("y0", cat="Binary")
ex2b_y1 = pl.LpVariable("y1", cat="Binary")
ex2b_y4 = pl.LpVariable("y4", cat="Binary")
ex2b_y6 = pl.LpVariable("y6", cat="Binary")
ex2b_w = pl.LpVariable("w")

# Função objetivo
ex2b_lp += ex2b_x1 + 2*ex2b_x2

# Restrições
ex2b_lp += ex2b_y0 + ex2b_y1 + ex2b_y4 + ex2b_y6 == 1
ex2b_lp += ex2b_y1 + 4*ex2b_y4 + 6*ex2b_y6 - ex2b_x1 == 0
ex2b_lp += ex2b_y1 + 16*ex2b_y4 + 36*ex2b_y6 - ex2b_w == 0
ex2b_lp += ex2b_x1 + ex2b_x2 <= 8
ex2b_lp += -ex2b_x1 + ex2b_x2 <= 2
ex2b_lp += ex2b_x1 - ex2b_x2 <= 4

# Resolver
ex2b_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex2b_lp.status])
for var in ex2b_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex2b_lp.objective))

Status: Optimal
w = 16.0
x1 = 4.0
x2 = 4.0
y0 = 0.0
y1 = 0.0
y4 = 1.0
y6 = 0.0
z = 12.0


## Exercício 3

Applied Mathematical Programming | Capítulo 9 - Exercício 3

### Enunciado

O grupo de marketing da A. J. Pitt Company está considerando as opções disponíveis para sua próxima campanha publicitária.  
Após um grande esforço de análise, o grupo identificou um número selecionado de opções com as características mostradas na tabela abaixo:

| Meio                 | TV (1)       | Revista técnica (2) | Jornal (3)   | Rádio (4)    | Revista popular (5) | Campanha promocional (6) | Recursos totais disponíveis |
|----------------------|----------|-----------------|----------|----------|-----------------|-----------------------|-----------------------------|
| **Clientes atingidos** | 1.000.000 | 200.000         | 300.000  | 400.000  | 450.000         | 450.000               | –                           |
| **Custo ($)**        | 500.000  | 150.000         | 300.000  | 250.000  | 250.000         | 100.000               | 1.800.000                   |
| **Designers necessários** (h.h.) | 700      | 250             | 200      | 200      | 300             | 400                   | 1.500                       |
| **Vendedores necessários** (h.h.) | 200      | 100             | 100      | 100      | 100             | 1.000                 | 1.200                       |

(h.h. = horas-homens)


O objetivo do programa de publicidade é maximizar o número de clientes atingidos, sujeito às limitações de recursos (dinheiro, designers e vendedores) mostradas na tabela acima.  

Além disso, devem ser satisfeitas as seguintes restrições:

1. Se a campanha promocional for realizada, será necessário também realizar rádio ou revista popular para apoiá-la.  
2. A empresa não pode anunciar simultaneamente em revista técnica e em revista popular.  

 
Formule um modelo de programação inteira que auxilie a empresa a selecionar uma estratégia adequada de campanha publicitária.

### Solução

Sejam $x_1, x_2, x_3, x_4, x_5, x_6$ variáveis binárias que indicam se as publicidades na TV, revista técnica, jornal, rádio, revista popular ou campanha promocional foram escolhidas, respectivamente. Nossa função objetivo é:

Maximizar: $z = 1000x_1 + 200x_2 + 300x_3 + 400x_4 + 450x_5 + 450x_6$ (em milhares de dólares)

Das restrições de recursos, temos:
- Custo
    $500x_1 + 150x_2 + 300x_3 + 250x_4 + 250x_5 + 100x_6 \leq 1800$
- Designers necessários
    $700x_1 + 250x_2 + 200x_3 + 200x_4 + 300x_5 + 400x_6 \leq 1500$
- Vendedores necessários
    $200x_1 + 100x_2 + 100x_3 + 100x_4 + 100x_5 + 1000x_6 \leq 1200$

Além disso, das outras restrições do problema, temos:
1. Se a campanha promocional for realizada, será necessário também realizar rádio ou revista popular para apoiá-la.  
    $x_4 + x_5 \geq x_6 \Rightarrow x_4 + x_5 - x_6 \geq 0$
2. A empresa não pode anunciar simultaneamente em revista técnica e em revista popular.
    $x_2 + x_5 \leq 1$


In [201]:
# Criação do problema
ex3_lp = pl.LpProblem("Advertising", pl.LpMaximize)

# Variáveis de decisão
ex3_x1 = pl.LpVariable("x1", cat="Binary")
ex3_x2 = pl.LpVariable("x2", cat="Binary")
ex3_x3 = pl.LpVariable("x3", cat="Binary")
ex3_x4 = pl.LpVariable("x4", cat="Binary")
ex3_x5 = pl.LpVariable("x5", cat="Binary")
ex3_x6 = pl.LpVariable("x6", cat="Binary")

# Função objetivo
ex3_lp += 1000*ex3_x1 + 200*ex3_x2 + 300*ex3_x3 + 400*ex3_x4 + 450*ex3_x5 + 450*ex3_x6

# Restrições
ex3_lp += 500*ex3_x1 + 150*ex3_x2 + 300*ex3_x3 + 250*ex3_x4 + 250*ex3_x5 + 100*ex3_x6 <= 1800
ex3_lp += 700*ex3_x1 + 250*ex3_x2 + 200*ex3_x3 + 200*ex3_x4 + 300*ex3_x5 + 400*ex3_x6 <= 1500
ex3_lp += 200*ex3_x1 + 100*ex3_x2 + 100*ex3_x3 + 100*ex3_x4 + 100*ex3_x5 + 1000*ex3_x6 <= 1200
ex3_lp += ex3_x4 + ex3_x5 - ex3_x6 >= 0
ex3_lp += ex3_x2 + ex3_x5 <= 1

# Resolver
ex3_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex3_lp.status])
for var in ex3_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex3_lp.objective))

Status: Optimal
x1 = 1.0
x2 = 0.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 0.0
z = 2150.0


## Exercício 4

Applied Mathematical Programming | Capítulo 8 - Exercício 2

### Enunciado

Em um determinado dia durante a temporada de turismo, uma empresa de aluguel de carros deve fornecer veículos para quatro destinos, de acordo com o seguinte cronograma:

| Destino | Carros necessários |
|---------|--------------------|
| A       | 2                  |
| B       | 3                  |
| C       | 5                  |
| D       | 7                  |

A empresa possui três filiais a partir das quais os carros podem ser fornecidos. Nesse dia, o inventário de cada filial era o seguinte:

| Filial | Carros disponíveis |
|--------|--------------------|
| 1      | 6                  |
| 2      | 1                  |
| 3      | 10                 |

As distâncias entre as filiais e os destinos são dadas na tabela a seguir:

| Filial \ Destino | A  | B  | C  | D  |
|------------------|----|----|----|----|
| 1                | 7  | 11 | 3  | 2  |
| 2                | 1  | 6  | 0  | 1  |
| 3                | 9  | 15 | 8  | 5  |



Planeje a atividade do dia de forma que as demandas sejam atendidas com custo mínimo (assumindo que o custo é proporcional a carros x milhas percorridas).


### Solução

Sejam $x_{ij}$ o número de carros da filial i que vão para o destino j e $d_{ij}$ a distância associada. Nossa função objetivo é:

Minimizar $z = \sum_{i=1}^m \sum_{j=1}^n d_{ij} x_{ij}$

Com as restrições:
- $\sum_{j=1}^n x_{ij} = a_i$, onde $a_i$ é o número de carros disponíveis na filial i.
- $\sum_{i=1}^m x_{ij} = b_j$, onde $b_j$ é o número de carros necessários no destino j.
- $x_{ij} \geq 0 (i=1,..,m; j=1,...,n)$

In [202]:
m = 3
n = 4

# Matriz de distâncias
d = [[7, 11, 3, 2],
    [1, 6, 0, 1],
    [9, 15, 8, 5]]

# Carros disponíveis
a = [6, 1, 10]

# Carros necessários
b = [2, 3, 5, 7]

# Criação do problema
ex4_lp = pl.LpProblem("Car_Rent", pl.LpMinimize)

# Variáveis de decisão
x = [[pl.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Continuous') for j in range(n)] for i in range(m)]

# Função objetivo
ex4_lp += pl.lpSum(d[i][j] * x[i][j] for i in range(m) for j in range(n))

# Restrições
for i in range(m):
    ex4_lp += pl.lpSum(x[i][j] for j in range(n)) == a[i]
for j in range(n):
    ex4_lp += pl.lpSum(x[i][j] for i in range(m)) == b[j]

# Resolver
ex4_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex4_lp.status])
for var in ex4_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex4_lp.objective))

Status: Optimal
x_0_0 = 0.0
x_0_1 = 1.0
x_0_2 = 5.0
x_0_3 = 0.0
x_1_0 = 0.0
x_1_1 = 1.0
x_1_2 = 0.0
x_1_3 = 0.0
x_2_0 = 2.0
x_2_1 = 1.0
x_2_2 = 0.0
x_2_3 = 7.0
z = 100.0


## Exercício 5

Applied Mathematical Programming | Capítulo 8 - Exercício 4

### Enunciado

Uma grande rede de artigos esportivos deseja comprar 300, 200, 150, 500 e 400 raquetes de tênis de cinco tipos diferentes. Foram recebidas propostas de quatro fabricantes, que fornecerão no máximo as seguintes quantidades (todos os cinco tipos de raquetes combinados):

- M1: 600  
- M2: 500  
- M3: 300  
- M4: 400  

A loja estima que o lucro por raquete variará de acordo com o fabricante, conforme mostrado abaixo:

| Fabricante | R1   | R2   | R3   | R4   | R5   |
|------------|------|------|------|------|------|
| M1         | 5.50 | 7.00 | 8.50 | 4.50 | 3.00 |
| M2         | 6.00 | 6.50 | 9.00 | 3.50 | 2.00 |
| M3         | 5.00 | 7.00 | 9.50 | 4.00 | 2.50 |
| M4         | 6.50 | 5.50 | 8.00 | 5.00 | 3.50 |

  
Como os pedidos devem ser distribuídos entre os fabricantes?


### Solução

Sejam $x_{ij}$ o número de raquetes do tipo i compradas com o fornecedor j e $c_{ij}$ o lucro associado. Nossa função objetivo é:

Maximizar $z = \sum_{i=1}^m \sum_{i=1}^n x_{ij} c_{ij}$

Com as restrições:
- $\sum_{i=1}^m x_{ij} \leq r_j$, onde $r_j$ é o máximo de raquetes fornecidas pelo fabricante j.
- $x_{ij} \geq 0$

In [203]:
m = 5
n = 4

# Matriz de lucros
d = [[5.5, 6, 5, 6.5],
    [7, 6.5, 7, 5.5],
    [8.5, 9, 9.5, 8],
    [4.5, 3.5, 4, 5],
    [3, 2, 2.5, 3.5]]

# Número de raquetes fornecidas
r = [600, 500, 300, 400]

# Criação do problema
ex5_lp = pl.LpProblem("Tennis_Racquets", pl.LpMaximize)

# Variáveis de decisão
x = [[pl.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Continuous') for j in range(n)] for i in range(m)]

# Função objetivo
ex5_lp += pl.lpSum(d[i][j] * x[i][j] for i in range(m) for j in range(n))

# Restrições
for j in range(n):
    ex5_lp += pl.lpSum(x[i][j] for i in range(m)) <= r[j]

# Resolver
ex5_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex5_lp.status])
for var in ex5_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex5_lp.objective))

Status: Optimal
x_0_0 = 0.0
x_0_1 = 0.0
x_0_2 = 0.0
x_0_3 = 0.0
x_1_0 = 0.0
x_1_1 = 0.0
x_1_2 = 0.0
x_1_3 = 0.0
x_2_0 = 600.0
x_2_1 = 500.0
x_2_2 = 300.0
x_2_3 = 400.0
x_3_0 = 0.0
x_3_1 = 0.0
x_3_2 = 0.0
x_3_3 = 0.0
x_4_0 = 0.0
x_4_1 = 0.0
x_4_2 = 0.0
x_4_3 = 0.0
z = 15650.0


## Exercício extra

Applied Mathematical Programming | Capítulo 9 - Exercício 15

#### Enunciado

Um time universitário de basquete de quatro jogadores está tentando escolher sua formação inicial a partir de um elenco de seis jogadores, de forma a maximizar a altura média. O elenco é o seguinte:

| Jogador | Número | Altura* | Posição  |
|---------|--------|---------|----------|
| Dave    | 1      | 10      | Pivô     |
| John    | 2      | 9       | Pivô     |
| Mark    | 3      | 6       | Ala      |
| Rich    | 4      | 6       | Ala      |
| Ken     | 5      | 4       | Armador  |
| Jim     | 6      | -1      | Armador  |

\* Altura em polegadas acima de 5'6''.


A formação inicial deve satisfazer as seguintes restrições:

1. Pelo menos um armador deve iniciar.  
2. Ou John ou Ken deve ficar na reserva.  
3. Apenas um pivô pode iniciar.  
4. Se John ou Rich iniciarem, então Jim não pode iniciar.  

#### Solução

Usaremos as variáveis binárias $x_i, i=1,..,6$ para indicar se o jogador $i$ começa ou não. Nossa função objetivo é:

Maximizar $z = 10x_1 + 9x_2 + 6x_3 + 6x_4 + 4x_5 - x_6$


Agora, vamos modelar cada uma das restrições:

1. Pelo menos um armador deve iniciar.

    $x_5 + x_6 \geq 1$

2. Ou John ou Ken deve ficar na reserva.

    $x_2 + x_5 = 1$ (considerando que é um ou exclusivo)

3. Apenas um pivô pode iniciar.

    $x_1 + x_2 \leq 1$ (considerando que no máximo um pivô pode iniciar)

4. Se John ou Rich iniciarem, então Jim não pode iniciar. 

    $x_2 + x_6 \leq 1$

    $x_4 + x_6 \leq 1$

5. Como o time é composto de quatro jogadores, temos:

    $x_1 + x_2 + x_3 + x_4 + x_5 - x_6 = 4$

In [204]:
# Criação do problema
ex_lp = pl.LpProblem("Basketball", pl.LpMaximize)

# Variáveis de decisão
ex_x1 = pl.LpVariable("x1", cat="Binary")
ex_x2 = pl.LpVariable("x2", cat="Binary")
ex_x3 = pl.LpVariable("x3", cat="Binary")
ex_x4 = pl.LpVariable("x4", cat="Binary")
ex_x5 = pl.LpVariable("x5", cat="Binary")
ex_x6 = pl.LpVariable("x6", cat="Binary")

# Função objetivo
ex_lp += 10*ex_x1 + 9*ex_x2 + 6*ex_x3 + 6*ex_x4 + 4*ex_x5 - ex_x6

# Restrições
ex_lp += ex_x5 + ex_x6 >= 1
ex_lp += ex_x2 + ex_x5 == 1
ex_lp += ex_x1 + ex_x2 <= 1
ex_lp += ex_x2 + ex_x6 <= 1
ex_lp += ex_x4 + ex_x6 <= 1
ex_lp += ex_x1 + ex_x2 + ex_x3 + ex_x4 + ex_x5 + ex_x6 == 4

# Resolver
ex_lp.solve(pl.PULP_CBC_CMD(msg=0))

# Solução
print("Status:", pl.LpStatus[ex_lp.status])
for var in ex_lp.variables():
    print(var, "=", var.varValue)
print("z =", pl.value(ex_lp.objective))

Status: Optimal
x1 = 1.0
x2 = 0.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 0.0
z = 26.0
